---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [93]:
import pandas as pd
import numpy as np
import re

def answer_one():    
    energy = pd.read_excel('Energy Indicators.xls', usecols = [2,3,4,5], skiprows = 17, skipfooter=38,
                               names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])

    # Convert Energy Supply to the same unit as Energy Supply per capita, 
    # Petajoules --> Gigajoules, there are 1,000,000 Gigajoules in a Petajoule
    energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x*1e6 if str(x).isnumeric() else np.nan)

    # Remove numbers and parens from Country names
    energy['Country'] = (energy['Country']
                             .apply(lambda x: x.split(' ')[0])
                             .apply(lambda x: re.sub(r'[0-9]+', '', x)))

    energy.set_index('Country', inplace=True)

    energy.rename(
        {"Republic of Korea": "South Korea",
         "United States of America": "United States",
         "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
         "China, Hong Kong Special Administrative Region": "Hong Kong"})

    GDP = (pd.read_csv('world_bank.csv', skiprows=4, index_col=0)
               .rename({"Korea, Rep.": "South Korea", 
                        "Iran, Islamic Rep.": "Iran",
                        "Hong Kong SAR, China": "Hong Kong"}))

    ScimEn = pd.read_excel('scimagojr-3.xlsx', index_col = 1)

    # Now need to join all three tables and result in a single table with the 
    # following columns
    #  ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
    # 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
    # '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

    df = ScimEn.join(energy, how = 'outer').join(GDP, how='outer')

    df = df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 
             'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable',
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    df = df[df['Rank'].apply(lambda x: x in range(1, 16))].sort_values('Rank')
    return(df)

# Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [94]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [95]:
def answer_two():
    return (energy.shape[0] + GDP.shape[0] + ScimEn.shape[0]) - (3*15)


## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [96]:
def answer_three():
    df = answer_one()
    year_cols = df.columns[df.columns.str.startswith('20')]
    avgGDP = (df.apply(lambda x: x[year_cols].mean(skipna = True), axis=1)
                  .sort_values(ascending = False))
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [97]:
def answer_four():
    df = answer_one()
    six = answer_three().index[5]
    answer = df.loc[six, '2015'] - df.loc[six, '2006']
    return answer

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [98]:
def answer_five():
    df = answer_one()
    answer = df['Energy Supply per Capita'].mean(skipna = True)
    return answer

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [99]:
def answer_six():
    df = answer_one()
    idx = df['% Renewable'].idxmax()
    answer = (idx, df.loc[idx, '% Renewable'])
    return answer

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [100]:
def answer_seven():
    df['Citation-Ratio'] = df['Self-citations'] / df['Citations']
    idx = df['Citation-Ratio'].idxmax()
    answer = (idx, df.loc[idx, 'Citation-Ratio'])
    return answer


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [101]:
def answer_eight():
    df = answer_one()
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita'] 
    answer = df['Pop by Energy'].sort_values(ascending = False).index[2]
    return answer

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [104]:
def answer_nine():
    df = answer_one()
    df['PopEst'] = Top15['Energy Supply'] / df['Energy Supply per Capita']
    df['Citable docs per Capita'] = df['Citable documents'] / df['PopEst']
    x = df.loc[df['Energy Supply per Capita'].notnull(), 'Energy Supply per Capita'].astype(float)
    y = df.loc[df['Citable docs per Capita'].notnull(), 'Citable docs per Capita'].astype(float)
    answer = x.corr(y)
    return answer


In [103]:
#def plot9():
#import matplotlib as plt
#%matplotlib inline
    
#Top15 = answer_one()
#Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
#Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
#Top15 = Top15.fillna(0)
#Top15['Energy Supply per Capita']
#Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])


### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [105]:
def answer_ten():
    df = answer_one()
    median = df['% Renewable'].median()
    HighRenew = (df['% Renewable']
                     .apply(lambda x: 1 if x > median else 0)
                     .sort_index(ascending = True))

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [108]:
def answer_eleven():
    Top15 = answer_one()
    return "ANSWER"

ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

df = answer_one()
df = df.groupby(ContinentDict)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Asia,7.0,38687.600000,38527.400000,224233.800000,110433.2,7.324,112.6,4.713550e+10,12.666133,2.417068e+12,2.593688e+12,2.686833e+12,2.745338e+12,2.961825e+12,3.105645e+12,3.246476e+12,3.401622e+12,3.554373e+12,4.525337e+12
Australia,14.0,8831.000000,8725.000000,90765.000000,15606.0,10.280,107.0,5.386000e+09,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Europe,8.0,15008.333333,14764.333333,124456.833333,26158.0,8.860,109.5,8.827750e+09,26.639408,2.227191e+12,2.300005e+12,2.317125e+12,2.204494e+12,2.258262e+12,2.306675e+12,2.307796e+12,2.314700e+12,2.341307e+12,2.366393e+12
North America,4.0,57280.000000,56183.500000,503638.500000,153183.0,10.105,189.5,1.043100e+10,61.945430,8.178386e+12,8.326068e+12,8.312102e+12,8.079993e+12,8.288889e+12,8.434053e+12,8.617647e+12,8.752177e+12,8.965052e+12,9.170591e+12
South America,15.0,8668.000000,8596.000000,60702.000000,14396.0,7.000,86.0,1.214900e+10,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!